In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
plt.style.use('seaborn')
sns.set(font_scale=2)
from collections import Counter

%matplotlib inline
import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torch
from torchvision import transforms
from PIL import Image

#CUDA 메모리 할당 / 디바이스 할당
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

C:\Users\PC\AppData\Local\Temp\ipykernel_22472\2823742555.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
# ResNet : residual block이 겹겹이 쌓여 구성된 모델
# residual block 정의

import torch.utils.data as data

class BasicDataset(data.Dataset):
  def __init__(self,data,label,transform):
    super(BasicDataset, self).__init__()
    self.data=data
    self.label=label
    self.transform=transform
  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    sample_X = self.data[idx]
    sample_y = self.label[idx]
    if self.transform:
      sample_X = self.transform(sample_X)
    return sample_X, sample_y

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # BatchNorm에 bias가 포함되어 있으므로, conv2d는 bias=False로 설정합니다.
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        # identity mapping, input과 output의 feature map size, filter 수가 동일한 경우 사용.
        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        # projection mapping using 1x1conv
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=10, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

# CIFAR10 Data load

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

transform_train = transforms.Compose([
        transforms.Normalize(mean, std),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding = 4)
    ])

transform = transforms.Compose([
        transforms.Normalize(mean, std)
    ])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
trainset = torchvision.datasets.CIFAR10(root='./data/vision-greg/cifarpy', train=True, download=True, transform=transform_train)

testset=torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testset=torchvision.datasets.CIFAR10(root='./data/vision-greg/cifarpy', train=False, download=True, transform=transform)

def resize_images(dataset, target_size=(64, 64)):
    
    num_samples, num_channels, _, _ = dataset.shape
    resized_dataset = np.zeros((num_samples, num_channels, target_size[0], target_size[1]), dtype=np.uint8)

    for i in range(num_samples):
        for channel in range(num_channels):
            # Create a PIL image from the channel data
            pil_image = Image.fromarray(dataset[i, channel])

            # Resize the image to the target size using PIL
            pil_image = pil_image.resize(target_size, Image.ANTIALIAS)

            # Convert the resized PIL image back to a NumPy array
            resized_dataset[i, channel] = np.array(pil_image)

    return resized_dataset
    
train_data = np.transpose(trainset.data, (0,3,1,2))
test_data = np.transpose(testset.data, (0,3,1,2))

train_data=resize_images(train_data, target_size = (64,64))
test_data=resize_images(test_data, target_size = (64,64))

In [4]:
# transform

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

transform_train = transforms.Compose([
        transforms.Normalize(mean, std),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding = 4)
    ])

transform = transforms.Compose([
        transforms.Normalize(mean, std)
    ])

# Load cifar10 data
train_X = np.load('cifar10_train_data.npy')
train_Y = np.load('cifar10_train_label.npy')

test_X = np.load('cifar10_test_data.npy')
test_Y = np.load('cifar10_test_label.npy')

# Into torch.Tensor
train_X = torch.Tensor(train_X)
train_Y = torch.Tensor(train_Y)

test_X = torch.Tensor(test_X)
test_Y = torch.Tensor(test_Y)

# 데이터셋 생성
dataset_train = BasicDataset(train_X, train_Y, transform=transform_train)
dataset_test = BasicDataset(test_X, test_Y, transform=transform)

In [5]:
test_X.shape

torch.Size([10000, 3, 64, 64])

# OOD

# data불러오기
#OOD = np.load('300K_random_images.npy')

# 순서변경
#OOD = np.transpose(OOD, (0,3,1,2))

# 32*32에서 64*64로 변경하기
#OOD = resize_images(OOD, target_size = (64,64))

OOD = np.load('300K_random_images_64.npy')
# torch.Tensor
#OOD = torch.Tensor(OOD)

# 임의의 Trarget Labels 만들기
desired_size = (len(OOD), 1, 10)
OOD_label = torch.zeros(desired_size)

# OOD와 OOD_label 합체하기
OOD_dataset = BasicDataset(OOD, OOD_label, transform = transform_train)

In [6]:
TinyImage = np.load('300K_random_images_64.npy')
sample_indices = np.random.choice(300000, 50000, replace = False)
OOD = torch.Tensor(TinyImage[sample_indices])

desired_size = (50000, 1, 10)
OOD_label = torch.zeros(desired_size)

OOD_dataset = BasicDataset(OOD, OOD_label, transform = transform_train)

# Model

In [7]:
model = resnet34().to(DEVICE)

args = {
	'BATCH_SIZE':64,
        'LEARNING_RATE': 0.001,
        'NUM_EPOCH': 20
        }

criterion = torch.nn.CrossEntropyLoss().to(DEVICE) # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=args['LEARNING_RATE'])
torch.manual_seed(42)

In [8]:
# 추가
class SaveBestModel:
    def __init__(
        self, best_train_loss=float('inf')
    ):
        self.best_train_loss = best_train_loss

    def __call__(
        self, current_train_loss,
        epoch, model, optimizer, criterion
    ):
        if current_train_loss < self.best_train_loss:
            self.best_train_loss = current_train_loss
            print(f"\nBest train loss: {self.best_train_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, 'outputs/best_model.pth')

save_best_model = SaveBestModel()

In [9]:
# 연습용
def train_epoch(model, train_loader_in, train_loader_out, loss_fn, optimizer):
    train_loss,train_correct=0.0,0
    model.train()
    
    #for images, labels in dataloader:
    for (in_set, out_set) in zip(train_loader_in, train_loader_out):
        data = torch.cat((in_set[0], out_set[0]), dim=0)
        target = in_set[1]
        
        data, target = data.to(DEVICE), target.to(DEVICE).reshape(args['BATCH_SIZE'], 10) # CIFAR10은 10으로 변경
        
        # forward
        x = model(data)
        
        # backward
        optimizer.zero_grad()
        
        target = torch.argmax(target, dim = 1)
        loss = loss_fn(x[:len(in_set[0])], target)
        # cross-entropy from softmax distribution to uniform distribution
        #loss += 0.5 * -(x[len(in_set[0]):].mean(1) - torch.logsumexp(x[len(in_set[0]):], dim=1)).mean()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * target.shape[0]
        predictions = torch.argmax(x[:len(in_set[0])], 1)
        
        train_correct += (predictions == target).sum().item()
        

    return train_loss, train_correct

def test_epoch(model, dataloader, loss_fn):
    test_loss, test_correct = 0.0, 0
    model.eval()
    y_true, y_pred = [], []
    for images, labels in dataloader:
        images,labels = images.to(DEVICE),labels.to(DEVICE).reshape(args['BATCH_SIZE'], 10)# CIFAR10은 10으로 변경
        output = model(images)
        labels = torch.argmax(labels, dim=1)
        loss=loss_fn(output, labels)
        test_loss += loss.item() * images.size(0)
        predictions = torch.argmax(output, 1)
        test_correct += (predictions == labels).sum().item()
        y_true += labels.tolist()
        y_pred += predictions.tolist()
    return test_loss, test_correct, y_true, y_pred

In [10]:
load = False # True 활용하면 model만 불러온다.
if load == False:
    train_loader_in = DataLoader(dataset_train, batch_size=args['BATCH_SIZE'], drop_last=True, shuffle = True)
    train_loader_out = DataLoader(OOD_dataset, batch_size=args['BATCH_SIZE'], drop_last = True, shuffle = True)
    history = {'train_loss': [], 'train_acc':[]}

    for epoch in range(args['NUM_EPOCH']):
      train_loss, train_correct=train_epoch(model,train_loader_in, train_loader_out, criterion,optimizer)
      train_loss = train_loss / len(train_loader_out.sampler)
      train_acc = train_correct / len(train_loader_out.sampler) * 100
    
      #print(train_correct)
      #print(len(train_loader_in.sampler))
    
      print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Training Acc: {:.2f} % ".format(epoch + 1, args['NUM_EPOCH'],train_loss ,train_acc))
      history['train_loss'].append(train_loss)
      history['train_acc'].append(train_acc)
      save_best_model(
          train_loss, epoch, model, optimizer, criterion
           )
    # load best model
    checkpoint = torch.load('outputs/best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])

    # test model using test_epoch function
    test_loader = DataLoader(dataset_test, batch_size=args['BATCH_SIZE'], drop_last=True)
    test_loss, test_correct, y_true, y_pred = test_epoch(model, test_loader, criterion)

    # calculate test set accuracy and loss
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.3f} Test Accuracy: {test_acc:.2f}%")
else:
    # load best model
    checkpoint = torch.load('outputs/best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])

    # test model using test_epoch function
    test_loader = DataLoader(dataset_test, batch_size=args['BATCH_SIZE'], drop_last=True)
    test_loss, test_correct, y_true, y_pred = test_epoch(model, test_loader, criterion)

    # calculate test set accuracy and loss
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.3f} Test Accuracy: {test_acc:.2f}%")

Epoch:1/20 AVG Training Loss:1.976 AVG Training Acc: 25.06 % 

Best train loss: 1.9759973809814453

Saving best model for epoch: 1

Epoch:2/20 AVG Training Loss:1.820 AVG Training Acc: 31.81 % 

Best train loss: 1.8203046623229981

Saving best model for epoch: 2

Epoch:3/20 AVG Training Loss:1.722 AVG Training Acc: 36.45 % 

Best train loss: 1.7215575477600098

Saving best model for epoch: 3

Epoch:4/20 AVG Training Loss:1.620 AVG Training Acc: 40.86 % 

Best train loss: 1.6196388858032227

Saving best model for epoch: 4

Epoch:5/20 AVG Training Loss:1.546 AVG Training Acc: 43.96 % 

Best train loss: 1.5460873715209962

Saving best model for epoch: 5

Epoch:6/20 AVG Training Loss:1.505 AVG Training Acc: 45.87 % 

Best train loss: 1.5050262826538086

Saving best model for epoch: 6

Epoch:7/20 AVG Training Loss:1.473 AVG Training Acc: 47.23 % 

Best train loss: 1.4733251556396485

Saving best model for epoch: 7

Epoch:8/20 AVG Training Loss:1.422 AVG Training Acc: 48.83 % 

Best train lo

KeyboardInterrupt: 

# Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45,fontsize = 10)
    plt.yticks(tick_marks, classes,fontsize = 10)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

test_loss, test_correct, y_true, y_pred = test_epoch(model, test_loader, criterion)

cm = confusion_matrix(y_true, y_pred)
classes = [str(i) for i in range(10)]
faulty_case  =['Airplane','Car','Bird','Cat','Deer','Dog','Frog','Horse','Ship','Truck']


# Plot the confusion matrix with larger font size
plt.figure(figsize=(8, 8))
plt.rcParams.update({'font.size': 10})  # 폰트 크기 조정
plot_confusion_matrix(cm, faulty_case, normalize=True, title='Normalized confusion matrix')
plt.show()

In [ ]:
sample_image = test_X

# 모델에 이미지 데이터 전달
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():  # 그라디언트 계산 비활성화
    output = model(sample_image.to(DEVICE))  # 모델에 이미지 데이터 전달

# 소프트맥스 함수를 적용하여 확률값 계산
softmax_output = torch.softmax(output, dim=1)

# 각 이미지별로 가장 큰 소프트맥스 값을 가진 값을 추출
softmax_in = torch.max(softmax_output, axis = 1).values
softmax_in.shape

In [ ]:
TPR95 = np.percentile(softmax_in.cpu().numpy(), q = 5)
print(TPR95)
plt.hist(softmax_in.cpu().numpy())
plt.axvline(x = TPR95, color = 'red')
plt.text(TPR95, 500, np.round(TPR95,3), fontsize=12, color='black')
plt.title('In Distribution max softmax score')

# Out of Distribution Data load

## 1. Mixed Donut

In [ ]:
group_out = np.load('mixed_wm811k/group_donut.npy')

group_out = torch.Tensor(group_out)

group_out[group_out == 255] = 1

sample_image = group_out

# 모델에 이미지 데이터 전달
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():  # 그라디언트 계산 비활성화
    output = model(sample_image.to(DEVICE))  # 모델에 이미지 데이터 전달

# 소프트맥스 함수를 적용하여 확률값 계산
softmax_output = torch.softmax(output, dim=1)

predicted_classes = torch.argmax(softmax_output, dim=1).tolist()

# 각 이미지별로 가장 큰 소프트맥스 값을 가진 값을 추출
softmax_out = torch.max(softmax_output, axis = 1).values
print(softmax_out.shape)
print(Counter(predicted_classes))

In [ ]:
plt.hist(softmax_out.cpu().numpy())
plt.axvline(x = TPR95, color = 'red')
plt.text(TPR95, 500, np.round(TPR95,3), fontsize=12, color='black')
plt.title('Mixed Donut max softmax score - FPR: {:.3f}'.format(np.mean(softmax_out.cpu().numpy() > TPR95)))

## 2. Mixed Center

In [ ]:
group_out = np.load('mixed_wm811k/group_center.npy')

group_out = torch.Tensor(group_out)

group_out[group_out == 255] = 1

sample_image = group_out

# 모델에 이미지 데이터 전달
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():  # 그라디언트 계산 비활성화
    output = model(sample_image.to(DEVICE))  # 모델에 이미지 데이터 전달

# 소프트맥스 함수를 적용하여 확률값 계산
softmax_output = torch.softmax(output, dim=1)

predicted_classes = torch.argmax(softmax_output, dim=1).tolist()

# 각 이미지별로 가장 큰 소프트맥스 값을 가진 값을 추출
softmax_out = torch.max(softmax_output, axis = 1).values
print(softmax_out.shape)
print(Counter(predicted_classes))

In [ ]:
plt.hist(softmax_out.cpu().numpy())
plt.axvline(x = TPR95, color = 'red')
plt.text(TPR95, 600, np.round(TPR95,3), fontsize=12, color='black')
plt.title('Mixed Center max softmax score - FPR: {:.3f}'.format(np.mean(softmax_out.cpu().numpy() > TPR95)))

## 3. Unlike Donut

In [ ]:
group_out = np.load('mixed_wm811k/group_unlike_donut.npy')

group_out = torch.Tensor(group_out)

group_out[group_out == 255] = 1

sample_image = group_out

# 모델에 이미지 데이터 전달
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():  # 그라디언트 계산 비활성화
    output = model(sample_image.to(DEVICE))  # 모델에 이미지 데이터 전달

# 소프트맥스 함수를 적용하여 확률값 계산
softmax_output = torch.softmax(output, dim=1)

predicted_classes = torch.argmax(softmax_output, dim=1).tolist()

# 각 이미지별로 가장 큰 소프트맥스 값을 가진 값을 추출
softmax_out = torch.max(softmax_output, axis = 1).values
print(softmax_out.shape)
print(Counter(predicted_classes))

In [ ]:
plt.hist(softmax_out.cpu().numpy())
plt.axvline(x = TPR95, color = 'red')
plt.text(TPR95, 800, np.round(TPR95,3), fontsize=12, color='black')
plt.title('Unlike Donut max softmax score - FPR: {:.3f}'.format(np.mean(softmax_out.cpu().numpy() > TPR95)))

# Error 잡기

In [ ]:
 for (in_set, out_set) in zip(train_loader_in, train_loader_out):
        data = torch.cat((in_set[0], out_set[0]), dim=0)
        target = in_set[1]
        
        data, target = data.to(DEVICE), target.to(DEVICE).reshape(args['BATCH_SIZE'], 8) # CIFAR10은 10으로 변경

In [ ]:
data.shape

In [ ]:
target.shape

In [ ]:
data.shape

In [ ]:
target.shape

In [ ]:
 for (in_set, out_set) in zip(train_loader_in, train_loader_out):
        data = torch.cat((in_set[0], out_set[0]), dim=0)
        target = in_set[1]
        
        data, target = data.to(DEVICE), target.to(DEVICE).reshape(args['BATCH_SIZE'], 8) # CIFAR10은 10으로 변경
        
        # forward
        x = model(data)
        
        # backward
        optimizer.zero_grad()
        
        target = torch.argmax(target, dim = 1)
        loss = criterion(x[:len(in_set[0])], target)
        # cross-entropy from softmax distribution to uniform distribution
        loss += 0.5 * -(x[len(in_set[0]):].mean(1) - torch.logsumexp(x[len(in_set[0]):], dim=1)).mean()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * target.shape[0]
        predictions = torch.argmax(x[:len(in_set[0])], 1)
        
        train_correct += (predictions == target).sum().item()

In [ ]:
x.shape

In [ ]:
x[len(in_set[0]):].mean(1)

In [ ]:
torch.logsumexp(x[len(in_set[0]):], dim=1)

In [ ]:
x[len(in_set[0]):].shape

In [ ]:
torch.logsumexp(x[len(in_set[0]):], dim=1).shape

In [ ]:
x.shape